# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv", encoding = "utf-8")
weather_data.head()

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
0,Bandarbeyla,9.4942,50.8122,72.14,72,13,14.07,SO,1612306357
1,Port Alfred,-33.5906,26.8910,66.00,91,13,9.24,ZA,1612306357
2,San Quintín,30.4833,-115.9500,65.62,61,87,15.08,MX,1612306357
3,Itarema,-2.9248,-39.9167,78.84,77,0,12.41,BR,1612306357
4,Fukuechō,32.6881,128.8419,42.98,66,100,18.63,JP,1612306358


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

In [12]:
fig = gmaps.figure(zoom_level=1.9, center=(25,0))

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# reduce dataframe to include only cities matching a certain criteria
# max temp>=75, <85
# wind speed<5mph
# cloudiness<= 5

reduced_wd = weather_data.loc[(weather_data["Max Temp. (F)"]>=75) &
                             (weather_data["Max Temp. (F)"]<85) &
                             (weather_data["Wind Speed (mph)"]<5) &
                              (weather_data["Cloudiness (%)"]<=15)]
reduced_wd

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
12,Cap Malheureux,-19.9842,57.6142,79.00,75,10,3.00,MU,1612306360
138,Grand Gaube,-20.0064,57.6608,79.00,75,12,3.00,MU,1612306388
169,Ndélé,8.4109,20.6473,77.86,24,8,2.26,CF,1612306396
222,Mtwara,-10.2667,40.1833,78.31,76,0,3.20,TZ,1612306408
434,Kavaratti,10.5669,72.6420,80.64,70,3,1.92,IN,1612306458
449,Lodwar,3.1191,35.5973,80.10,40,0,4.29,KE,1612306461
475,Harper,4.3750,-7.7169,78.37,89,10,4.54,LR,1612306467
481,Luganville,-15.5333,167.1667,84.52,67,8,1.86,VU,1612306468
516,Wamba,8.9333,8.6000,77.76,18,0,4.29,NG,1612306476


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = reduced_wd.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
12,Cap Malheureux,-19.9842,57.6142,79.00,75,10,3.00,MU,1612306360,
138,Grand Gaube,-20.0064,57.6608,79.00,75,12,3.00,MU,1612306388,
169,Ndélé,8.4109,20.6473,77.86,24,8,2.26,CF,1612306396,
222,Mtwara,-10.2667,40.1833,78.31,76,0,3.20,TZ,1612306408,
434,Kavaratti,10.5669,72.6420,80.64,70,3,1.92,IN,1612306458,
449,Lodwar,3.1191,35.5973,80.10,40,0,4.29,KE,1612306461,
475,Harper,4.3750,-7.7169,78.37,89,10,4.54,LR,1612306467,
481,Luganville,-15.5333,167.1667,84.52,67,8,1.86,VU,1612306468,
516,Wamba,8.9333,8.6000,77.76,18,0,4.29,NG,1612306476,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
